### The code and data are adapted from:  https://medium.com/@vitalshchutski/french-nlp-entamez-le-camembert-avec-les-librairies-fast-bert-et-transformers-14e65f84c148

In [ ]:
# !conda install torch
# !pip install fast-bert==1.9.1
# !mkdir model
# !mkdir finetuned_model
# manully download & install apex => https://github.com/NVIDIA/apex

In [1]:
import torch
from fast_bert.data_cls import BertDataBunch 
from fast_bert.learner_cls import BertLearner
from fast_bert.data_lm import BertLMDataBunch
from fast_bert.learner_lm import BertLMLearner
from fast_bert.metrics import fbeta, roc_auc
from fast_bert.prediction import BertClassificationPredictor
from pathlib import Path
import pandas as pd
import logging


device_cuda = torch.device("cuda")

### Fixing shitty windows encodings => failed 

In [2]:
import sys
sys.getdefaultencoding()

'utf-8'

In [3]:
import locale; 
if locale.getpreferredencoding().upper() != 'UTF-8': 
    locale.setlocale(locale.LC_ALL, ('fr', 'utf-8'))
locale.getpreferredencoding()

'UTF-8'

In [4]:
sys.getfilesystemencoding()

'utf-8'

### Data preparation

In [5]:
DATA_PATH = Path('./data/')
LOG_PATH = Path('./logs/')
MODEL_PATH = Path('./model/')
LABEL_PATH = Path('./labels/')

In [6]:
#create logger
logfile = str('logfile.txt')

logging.basicConfig(
    level=logging.INFO,  #CRITICAL ERROR WARNING  INFO    DEBUG    NOTSET
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%m/%d/%Y %H:%M:%S',
    handlers=[
        logging.FileHandler(logfile, 'w', 'utf-8'),
        logging.StreamHandler(sys.stdout)
    ])

logger = logging.getLogger()

In [7]:
df = pd.read_csv('./data/AA_xml_manul - 0-200_0-403.csv',encoding = 'utf-8')

In [8]:
#remove NaN
df = df[~df['Disease'].isnull()]


In [29]:
import numpy as np
df['Disease'] = df['Disease'].apply(np.int64)

In [9]:
df.tail(10)

,report_text,source_name,Bioagressor,Disease
392,doryphore des larves de doryphores ont été obs...,AA_PDT_Ile_de_France_2006_005,1,0.0
393,puceronson note l'apparition des premiers puce...,AA_PDT_Ile_de_France_2006_005,0,0.0
394,détermination du seuil de traitement :réalisez...,AA_PDT_Ile_de_France_2006_005,0,0.0
395,protection du maïs semence édité avec la colla...,AA_GC_Midi_Pyrenees_1993_013,0,0.0
396,situation il est possible d'observer les flétr...,AA_GC_Midi_Pyrenees_1993_013,1,0.0
397,préconisation toute intervention est aujourd'h...,AA_GC_Midi_Pyrenees_1993_013,0,0.0
398,situation - prévision le vol est réalisé à plu...,AA_GC_Midi_Pyrenees_1993_013,0,0.0
399,préconisationune intervention ne se justifiera...,AA_GC_Midi_Pyrenees_1993_013,0,0.0
400,situation - prévisiondes colonies de metopolop...,AA_GC_Midi_Pyrenees_1993_013,0,0.0
401,préconisationaucune intervention n'est nécessa...,AA_GC_Midi_Pyrenees_1993_013,0,0.0


In [31]:
val_set = df.sample(frac=0.2, replace=False, random_state=42)
train_set = df.drop(index = val_set.index)
print('Nombre de commentaires dans le val_set:',len(val_set))
print('Nombre de commentaires dans le train_set:', len(train_set))
val_set.to_csv('./data/val_set.csv',encoding = 'utf-8')
train_set.to_csv('./data/train_set.csv',encoding = 'utf-8')

Nombre de commentaires dans le val_set: 80
Nombre de commentaires dans le train_set: 321


In [12]:
labels = df.columns[2:4].to_list() 
with open('./labels/labels.txt', 'w',encoding = 'utf-8') as f:
    for i in labels:
        f.write(i + "\n")

In [17]:
#df_texts = pd.read_csv('./data/bsv_chunk256_raw_1001-1200.csv',encoding = 'utf-8')
#all_texts = df_texts['report_text'].to_list()
#all_texts = [str(t.encode('utf-8')) for t in all_texts] #that's the key to make it work under windows! => mais c'est diable
#print('Nombre de bloc de texte:', len(all_texts))

Nombre de bloc de texte: 2801


In [10]:
df_texts = pd.read_csv('./data/raw_xml_bsv_0-200.csv')

### text cleaning

In [11]:
import nltk
import re

# make all elements string
df_texts['report_text'] = df_texts['report_text'].astype(str)
# Remove null fields
df_texts['report_text'] = df_texts['report_text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# Make all text lowercase
df_texts['report_text'] = df_texts['report_text'].apply(lambda x: x.lower())
# Delete stop-words => to be tesred later
#stopwords = nltk.corpus.stopwords.words('french')


In [12]:
all_texts = df_texts['report_text'].to_list()[500:]
print('Nombre de bloc de texte:', len(all_texts))

Nombre de bloc de texte: 2297


In [13]:
all_texts[-9:]

['pucerons cendresintervenir avec aphicide nécessaire plus colonies',
 'betteraves',
 'pucerons jaunisse',
 'situationles premiers pucerons sont capturés bacs jaunes.',
 "préconisationles betteraves sont sensibles stade feuilles jusqu'à couverture sol. premier traitement sera effectuer prochain réchauffement particulièrement parcelles n'ayant reçu microgranulés semis ayant atteint stade feuilles. utiliser spécialité base pyréthrinoïde.",
 'principaux aphicides foliaires homologues betteraves* nouveaux produits contact, inhalation, ingestion',
 "note commune acta agpm inra atrazinel'atrazine désherbant utilisé france essentiellement cultures maïs pour l'entretien zones cultivées (voies ferrées, bordures routes, berges, etc...). suite utilisations, quelquefois observé résidus d'atrazine dans eaux supérieurs normes communautaires. donc souhaitable cette campagne promouvoir conditions d'emploi ('atrazjne visant modérer apports.",
 "culture mais-» eviter applications prélevée. préférer trai

### Création de LMDataBunch

In [14]:
databunch_lm = BertLMDataBunch.from_raw_corpus(
                    data_dir=DATA_PATH,
                    text_list=all_texts,
                    tokenizer='camembert-base',
                    batch_size_per_gpu=4, #was 16
                    max_seq_length=256, #was 512
                    multi_gpu=False,
                    model_type='camembert-base',
                    logger=logger)

12/10/2020 14:52:08 - INFO - root -   Formatting corpus for data\lm_train.txt


12/10/2020 14:52:08 - INFO - root -   Formatting corpus for data\lm_val.txt


12/10/2020 14:52:09 - INFO - transformers.tokenization_utils_base -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/camembert-base-sentencepiece.bpe.model from cache at C:\Users\Raja/.cache\torch\transformers\3715e3a4a2de48834619b2a6f48979e13ddff5cabfb1f3409db689f9ce3bb98f.28d30f926f545047fc59da64289371eef0fbdc0764ce9ec56f808a646fcfec59
12/10/2020 14:52:09 - INFO - root -   Loading features from cached file data\lm_cache\cached_camembert-base_train_256
12/10/2020 14:52:09 - INFO - root -   Loading features from cached file data\lm_cache\cached_camembert-base_dev_256


### Création de LMLearner

In [16]:
lm_learner = BertLMLearner.from_pretrained_model(
                            dataBunch=databunch_lm,
                            pretrained_path='camembert-base',
                            output_dir=MODEL_PATH,
                            metrics=[],
                            device=device_cuda,
                            logger=logger,
                            multi_gpu=False,
                            logging_steps=50,
                            is_fp16=False) #was true with gpu

12/10/2020 14:53:34 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/camembert-base-config.json from cache at C:\Users\Raja/.cache\torch\transformers\5152a7b8b97da26abdad9b3babb600e77c52a002331ea52a9eaf96ea8b31ef8f.5bd7a9a60b9a2d311368226259eaf870cfb2248e0752f28b444ec112977cf8fc
12/10/2020 14:53:34 - INFO - transformers.configuration_utils -   Model config CamembertConfig {
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "eos_token_id": 6,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 32005
}

12/10

In [17]:
lm_learner.fit(epochs=10, #was 30
            lr=1e-4,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")

12/10/2020 14:53:57 - INFO - root -   ***** Running training *****
12/10/2020 14:53:57 - INFO - root -     Num examples = 961
12/10/2020 14:53:57 - INFO - root -     Num Epochs = 10
12/10/2020 14:53:57 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 4
12/10/2020 14:53:57 - INFO - root -     Gradient Accumulation steps = 1
12/10/2020 14:53:57 - INFO - root -     Total optimization steps = 2410


12/10/2020 14:54:14 - INFO - root -   Running evaluation
12/10/2020 14:54:14 - INFO - root -   Num examples = 99
12/10/2020 14:54:14 - INFO - root -   Validation Batch size = 8


12/10/2020 14:54:15 - INFO - root -   eval_loss after step 50: 0.23925103705662948: 
12/10/2020 14:54:15 - INFO - root -   eval_perplexity after step 50: 1.2702974081039429: 
12/10/2020 14:54:15 - INFO - root -   lr after step 50: 9.989383241845838e-05
12/10/2020 14:54:15 - INFO - root -   train_loss after step 50: 2.7681958866119385


C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\torch\optim\lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


12/10/2020 14:54:31 - INFO - root -   Running evaluation
12/10/2020 14:54:31 - INFO - root -   Num examples = 99
12/10/2020 14:54:31 - INFO - root -   Validation Batch size = 8


...

12/10/2020 15:08:01 - INFO - root -   eval_loss after step 2300: 0.1509187542475187: 
12/10/2020 15:08:01 - INFO - root -   eval_perplexity after step 2300: 1.1629021167755127: 
12/10/2020 15:08:01 - INFO - root -   lr after step 2300: 5.131528823220099e-07
12/10/2020 15:08:01 - INFO - root -   train_loss after step 2300: 1.4004107570648194
12/10/2020 15:08:18 - INFO - root -   Running evaluation
12/10/2020 15:08:18 - INFO - root -   Num examples = 99
12/10/2020 15:08:18 - INFO - root -   Validation Batch size = 8


12/10/2020 15:08:20 - INFO - root -   eval_loss after step 2350: 0.14751078073795026: 
12/10/2020 15:08:20 - INFO - root -   eval_perplexity after step 2350: 1.1589457988739014: 
12/10/2020 15:08:20 - INFO - root -   lr after step 2350: 1.5285750326325954e-07
12/10/2020 15:08:20 - INFO - root -   train_loss after step 2350: 1.3645135688781738
12/10/2020 15:08:36 - INFO - root -   Running evaluation
12/10/2020 15:08:36 - INFO - root -   Num examples = 99
12/10/2020 15:08:36 - INFO - root -   Validation Batch size = 8


12/10/2020 15:08:38 - INFO - root -   eval_loss after step 2400: 0.14972286614087912: 
12/10/2020 15:08:38 - INFO - root -   eval_perplexity after step 2400: 1.1615122556686401: 
12/10/2020 15:08:38 - INFO - root -   lr after step 2400: 4.248146830060362e-09
12/10/2020 15:08:38 - INFO - root -   train_loss after step 2400: 1.3905822920799256
12/10/2020 15:08:41 - INFO - root -   Running evaluation
12/10/2020 15:08:41 - INFO - root -   Num examples = 99
12/10/2020 15:08:41 - INFO - root -   Validation Batch size = 8


12/10/2020 15:08:42 - INFO - root -   eval_loss after epoch 10: 0.14774326120431608: 
12/10/2020 15:08:42 - INFO - root -   eval_perplexity after epoch 10: 1.1592152118682861: 
12/10/2020 15:08:42 - INFO - root -   lr after epoch 10: 0.0
12/10/2020 15:08:42 - INFO - root -   train_loss after epoch 10: 1.363910719319498
12/10/2020 15:08:42 - INFO - root -   



(2410, 1.6330479007538918)

In [18]:
lm_learner.validate()

12/10/2020 15:22:29 - INFO - root -   Running evaluation
12/10/2020 15:22:29 - INFO - root -   Num examples = 99
12/10/2020 15:22:29 - INFO - root -   Validation Batch size = 8


{'loss': 0.14940655346100146, 'perplexity': 1.1611449718475342}

In [19]:
lm_learner.save_model()

12/10/2020 15:23:16 - INFO - transformers.configuration_utils -   Configuration saved in model\model_out\config.json
12/10/2020 15:23:18 - INFO - transformers.modeling_utils -   Model weights saved in model\model_out\pytorch_model.bin


In [20]:
del lm_learner

### Création de databunch pour la classification

In [21]:
databunch = BertDataBunch(DATA_PATH, LABEL_PATH,
                          tokenizer='camembert-base',
                          train_file='train_set.csv',
                          val_file='val_set.csv',
                          label_file='labels.txt',
                          text_col='report_text',
                          label_col=['Bioagressor','Disease'],
                          batch_size_per_gpu=8,
                          max_seq_length=256,
                          multi_gpu=False,
                          multi_label=True,
                          model_type='camembert-base')

12/10/2020 15:23:46 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/camembert-base-config.json from cache at C:\Users\Raja/.cache\torch\transformers\5152a7b8b97da26abdad9b3babb600e77c52a002331ea52a9eaf96ea8b31ef8f.5bd7a9a60b9a2d311368226259eaf870cfb2248e0752f28b444ec112977cf8fc
12/10/2020 15:23:46 - INFO - transformers.configuration_utils -   Model config CamembertConfig {
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "eos_token_id": 6,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 32005
}

12/10

### Création de Learner

In [22]:
metrics = [{'name': 'fbeta', 'function': fbeta}, {'name': 'roc_auc', 'function': roc_auc}]
OUTPUT_DIR = Path('./finetuned_model')
WGTS_PATH = Path('model/model_out/pytorch_model.bin')

In [23]:
# issue fast-bert pos_weight <= downgrade to 1.9.1 solve the prob
cl_learner = BertLearner.from_pretrained_model(
                        databunch,
                        pretrained_path='model/model_out',
                        metrics=metrics,
                        device=device_cuda, #was device_cuda
                        logger=logger,
                        output_dir=OUTPUT_DIR,
                        finetuned_wgts_path=WGTS_PATH,
                        warmup_steps=300,
                        multi_gpu=False,
                        multi_label=True,
                        is_fp16=False,#True when is cuda
                        logging_steps=50)

12/10/2020 15:24:20 - INFO - transformers.configuration_utils -   loading configuration file model/model_out\config.json
12/10/2020 15:24:20 - INFO - transformers.configuration_utils -   Model config CamembertConfig {
  "architectures": [
    "CamembertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 5,
  "eos_token_id": 6,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "camembert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 32005
}

12/10/2020 15:24:20 - INFO - transformers.modeling_utils -   loading weights file model/model_out\pytorch_model.bin
12/10/2020 15:24:22 - WARNING - transformers.modeling_utils -   Some weights of the model checkpoint at model/model_out were

In [24]:
cl_learner.fit(epochs=10,# was 30
            lr=2e-5,
            validate=True,
            schedule_type="warmup_cosine",
            optimizer_type="adamw")
#WHY??????????????????????????????????

12/10/2020 15:24:46 - INFO - root -   ***** Running training *****
12/10/2020 15:24:46 - INFO - root -     Num examples = 322
12/10/2020 15:24:46 - INFO - root -     Num Epochs = 10
12/10/2020 15:24:46 - INFO - root -     Total train batch size (w. parallel, distributed & accumulation) = 8
12/10/2020 15:24:46 - INFO - root -     Gradient Accumulation steps = 1
12/10/2020 15:24:46 - INFO - root -     Total optimization steps = 410


12/10/2020 15:24:52 - INFO - root -   Running evaluation
12/10/2020 15:24:52 - INFO - root -     Num examples = 80
12/10/2020 15:24:52 - INFO - root -     Batch size = 16


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [27]:
cl_learner.validate()

{'loss': 0.688255774974823, 'fbeta': 0.4375, 'roc_auc': 0.42315026697177727}

In [28]:
cl_learner.save_model()

In [37]:
del cl_learner

### Prédictions

In [30]:
predictor = BertClassificationPredictor(
                model_path='finetuned_model/model_out',
                label_path='labels/',
                multi_label=True,
                model_type='camembert-base',
                do_lower_case=False)

In [31]:
#cas disease: 0, bioagressor: 1 - cicadelle
predictor.predict("Cicadelles La cicadelle Edwardsiana est toujours observée sur les parcelles en été")

C:\Users\Raja\Documents\SJI\tools\anaconda3\envs\fast-camembert\lib\site-packages\pandas\core\frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,


[('disease', 0.336669921875), ('bioagressor', 0.277587890625)]

In [32]:
predictor.predict("election américane")

[('disease', 0.33740234375), ('bioagressor', 0.280029296875)]

In [33]:
#cas disease: 1, bioagressor: 0 - mildiou
predictor.predict("vigilant : en particulier vis-à-vis du mildiou, de l’oïdium et de la bactériose / cladosporiose. Les prévisions pour les prochains jours restent peu favorables à l’expression de la bactériose et la cladosporiose, si elles se confirment. Mildiou (Pseudoperonospora cubensis) : Le modèle annonce un risque élevé pour toutes les dates de plantation avec les données de la station de Thurageau. Avec les données de la station de Maulay, les plantations en S25 et S26 montrent un risque modéré. Le risque est un peu plus élevé dans le sud de la Charente- Maritime que dans le Poitou. Niveau de risque Faible Moyen Élevé Très élevé Indice : Log (Nb de taches/unité de surface) -14 à -9 -9 à -4 -4 +4 Équivalent en unité de surface 1 tâche par hectare par 100 m2 1 tâche par 100 m2 par m2 1 tâche par m2à 1 % de surface atteinte 1 % à 100 % de surface atteinte Évaluation du risque : les conditions restent favorables à ce microorganisme (qui n’est pas un champignon, mais proche d’une algue). Les BSV sont disponibles en accès direct sur le site  (rubrique : Nos publications - Bulletin de santé du végétal) ou par abonnement en ligne gratuit sur le site  BSV CULTURES LÉGUMIÈRES DE PLEIN")

[('disease', 0.33935546875), ('bioagressor', 0.283935546875)]

In [34]:
#cas disease: 0, bioagressor: 0 - texte sur trump trump
predictor.predict("L'avance de Donald Trump dans cet Etat où 4,9 millions d'électeurs ont voté, a fondu vendredi 6 novembre. Le candidat républicain compte")

[('disease', 0.3359375), ('bioagressor', 0.27490234375)]

In [35]:
#cas disease: 0, bioagressor: 0 - texte sur mouche de carotte mais pas de rique
predictor.predict("mouche de la carotte :ajouter trichloronate")

[('disease', 0.33544921875), ('bioagressor', 0.280029296875)]

In [29]:
#cas disease: 0, bioagressor: 0 - oidium : pas d'intervention => pas bon
predictor.predict("oïdium : pas d'intervention dans l'immediat les conditions très chaudes et l'absence de rosées nocturnes sont défavorables à cette maladie. aucun symptôme n'a encore été observé.")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [30]:
#cas disease: 1, bioagressor: 0 - oidium  => good
predictor.predict("oïdium du pommierce champignon est à l'heure actuelle en pleine fructification. il est nécessaire d'ajouter un antioïdium aux bouillies pour assurer'la protection du jeune feuillage.")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [31]:
#cas disease: 0, bioagressor: 0 - mildiou : pas d'intervention :=> pas bon
predictor.predict("mildiou : pas d'intervention dans l'immediat les conditions très chaudes et l'absence de rosées nocturnes sont défavorables à cette maladie. aucun symptôme n'a encore été observé.")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [32]:
#cas disease: 0, bioagressor: 0 - pucerons- le seuil de nuisibilité  n’est pas atteint => raté
predictor.predict("pucerons les pucerons verts sont toujours présents dans certaines parcelles. on note également la présence de sitobion avenae, relativement moins fréquente, mais avec des colonies de plusieurs dizaines d'individus. le temps frais actuel n'est pas favorable aux prédateurs, mais on observe un nombre important de pucerons «momifiés» par la ponte de parasitoïdes. le seuil de nuisibilité n'est atteint dans aucune parcelle du réseau")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [33]:
#cas disease: 0, bioagressor: 0 - zero mot clé
predictor.predict("par jour. Sur les autres secteurs, en production de carotte, les captures sont nulles. Évaluation du risque")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [34]:
#cas disease: 0, bioagressor: 0 + mildiou
predictor.predict("7 Action pilotée par le ministère chargé de l’agriculture mildiou, avec l’appui financier de l’Office National de l’Eau et des Milieux Aquatiques (ONEMA), par les crédits issus de la redevance pour pollutions diffuses attribués au")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [35]:
#cas disease: 0, bioagressor: 0
predictor.predict("financement du plan Ecophyto. Ce bulletin est rédigé par l'ACPEL avec la collaboration de référents par culture (techniciens des Chambres d'Agriculture de la Charente, de la Charente-Maritime et d'Indre et Loire et de la Vienne) sur la base d'observations réalisées par des producteurs et techniciens : Charentes- Alliance, les entreprises de production de melon, la coopérative AGROLEG, la coopérative UNIRE, des producteurs d'Agrobio Poitou-Charentes. Ce bulletin est réalisé à partir d'observations ponctuelles. Il a pour vocation de donner une tendance de la situation sanitaire régionale. Celle-ci ne peut être transposée telle quelle dans les parcelles de production légumières (conditions très variables). La Chambre Régionale d'Agriculture de Poitou-Charentes et le rédacteur dégagent toute responsabilité quant aux décisions prises par les producteurs pour la protection de leurs cultures. Elle les invite à prendre ces décisions sur la base des observation s qu'ils auront réalisées dans leurs parcelles. Les")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [36]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Ensemble de plantes cultivées pour leurs fruits ou leurs graines riches en matières grasses (lipides). De ces fruits et graines sont extrait une huile à usage alimentaire humaine, alimentaire animal ou industriel. Les résidus de l'extraction constituent les tourteaux utilisés pour l'alimentation animale.")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [37]:
#cas disease: 0, bioagressor: 0 - french crop usasge
predictor.predict("Orge semé après le 1er février, principalement de mars à mai. Orge de printemps est toujours à deux rangs")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [38]:
#cas disease: 1, bioagressor: 0 - tweet - jaunisse
predictor.predict("des parcelles qui ont eu une croissance presque nulle depuis cet été en majeur partie dû à la jaunisse.")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [39]:
#cas disease: 1, bioagressor: 0 - tweet - sécheresse
predictor.predict("Parfois,on nous demande. Que faites-vous,par cette sécheresse?")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [40]:
#cas disease: 0, bioagressor: 1 - tweet - pucerons

predictor.predict("Attention pucerons dans les blés")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]

In [41]:
predictor.predict("Favorisée par les conditions climatiques,l'activité du Carpocapse est toujours intense et il est conseillé de renouveler la protection des fruits en effectuant un nouveau traitement dès réception de ce bulletin.")

[('disease', 0.1798095703125), ('bioagressor', 0.0885009765625)]